In [1]:
import warnings
import logging


warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from utils import make_environments
from utils import pearl_utils
from configs  import defaults

from utils.reward_functions import log_reward_function,cumulative_reward_function,sharpe_reward_function
from utils. utils import make_hidden_dims
import optuna
from optuna.samplers import TPESampler
from neuralforecast.core import NeuralForecast
from Pearl.pearl.utils.instantiations.environments.gym_environment import GymEnvironment
from Pearl.pearl.utils.functional_utils.train_and_eval.online_learning import online_learning
import datetime

import numpy as np
import pickle
import boto3

ModuleNotFoundError: No module named 'futures_defaults'

In [1]:
from configs import defaults

ModuleNotFoundError: No module named 'futures_defaults'

In [3]:
# model=NeuralForecast.load('MultiHeadForecastingModel/')

In [ ]:
reward_functions=[log_reward_function,cumulative_reward_function,sharpe_reward_function]
train_env,test_env=make_environments.make_envs(reward_function=log_reward_function)
study_name=f"{defaults.model_name}-hp-search"
study_path="sqlite:///pearl_hyper_parameters.sqlite3"

FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

In [ ]:
today=datetime.datetime.now().strftime('%Y-%m-%d')
study = optuna.create_study(study_name=study_name,
                            directions=["maximize", "maximize"],
                            storage=study_name,
                            load_if_exists=True,
                            sampler=TPESampler()
                            )


[I 2024-11-17 11:37:39,585] Using an existing study with name 'pearl-2024-11-12-hp-search' instead of creating a new one.


In [12]:
# print(f"Best value: {study.best_value} (params: {study.best_params})")
best_trials=study.best_trials
best_trials

[FrozenTrial(number=16, state=TrialState.COMPLETE, values=[1084.2016722520668, 117.84], datetime_start=datetime.datetime(2024, 11, 12, 13, 32, 22, 388417), datetime_complete=datetime.datetime(2024, 11, 12, 13, 35, 46, 59501), params={'reward_function': 2, 'algorithm': 'ddqn', 'n_layers': 2, 'n_units': 128, 'training_rounds': 13, 'learning_rate': 3.53300011287604e-05, 'discount_factor': 0.8516537625726973, 'batch_size': 128, 'target_update_freq': 5, 'soft_update_tau': 0.6916880729632867, 'is_conservative': True, 'use_lstm': False, 'conservative_alpha': 0.9985474810108794, 'learn_after_episode': False, 'learning_steps': 10, 'n_epochs': 500}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'reward_function': CategoricalDistribution(choices=(0, 1, 2)), 'algorithm': CategoricalDistribution(choices=('dqn', 'ddqn')), 'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units': CategoricalDistribution(choices=(64, 128, 256, 512)), 'training_rounds': IntDistr

In [ ]:


if -1 in [np.sign(p) for p in test_env.positions]:
    market_type='Futures'
else:
    market_type='Spot'
symb=defaults.target_pair.replace('/','')

agent_path=f'Agent/pearl_{defaults.model_name}_model.pkl'

agent_path


'Agent/pearl_DOGEUSDT_Spot_model.pkl'

In [15]:
[i for i in range(5)]

[0, 1, 2, 3, 4]

In [ ]:
best_agents={}
for i in range(4):
    agent,learning_params=pearl_utils.load_agent_from_study(study_path=study_path,
                                            study_name=study_name,
                                            action_space_dim=2,
                                            observation_space_dim=30,
                                            version=i)
    
    agent,profit,n_trades=pearl_utils.train_production_agent(agent,
                                learning_params,
                                train_env=train_env,
                                test_env=train_env,
                                save_path=agent_path)
    best_agents[profit]=agent
agent=best_agents[max(best_agents)]
    

[I 2024-11-17 11:37:54,630] Using an existing study with name 'pearl-2024-11-12-hp-search' instead of creating a new one.
  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/84000 [00:00<?, ?it/s]

  0%|          | 0/500 [03:43<?, ?it/s]


Testing Return AVG Profit: 999.9449999999998, AVG Number of Trades: 0.55


  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:22<?, ?it/s]


Testing Return AVG Profit: 1012.5094668124718, AVG Number of Trades: 1.9


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:22<?, ?it/s]


In [17]:
# agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
#                                             study_name='pearl-2024-11-12-hp-search',
#                                             action_space_dim=2,
#                                             observation_space_dim=30,
#                                             version=1)

# agent=pearl_utils.train_production_agent(agent,
#                              learning_params,
#                              train_env=train_env,
#                              test_env=train_env,
#                              save_path=agent_path)

In [24]:
best_agents

{999.9449999999998: <Pearl.pearl.pearl_agent.PearlAgent at 0x345189e70>,
 999.9489999999996: <Pearl.pearl.pearl_agent.PearlAgent at 0x345bf86a0>,
 999.9539999999996: <Pearl.pearl.pearl_agent.PearlAgent at 0x3a32b46d0>,
 1012.5094668124718: <Pearl.pearl.pearl_agent.PearlAgent at 0x3a7f84f10>}

In [18]:
profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
print(f"Testing Return AVG Profit: {profit}, AVG Number of Trades: {n_trades}")

Testing Return AVG Profit: 1284.8120247137942, AVG Number of Trades: 34.08


In [ ]:
pickle.dump(agent,open(agent_path,'wb'))

import Keys

s3=boto3.client('s3', aws_access_key_id=Keys.AWS_ACCESS_KEY, aws_secret_access_key=Keys.AWS_SECRET_KEY)
s3.upload_file(agent_path,'coinbasetradehistory',agent_path.split('/')[-1],)

agent=pickle.load(open(agent_path,'rb'))

In [20]:
# pickle.dump(agent.policy_learner.state_dict(),open(agent_path,'wb'))

In [21]:
# weights=pickle.load(open(agent_path,'rb'))

In [22]:
# agent.policy_learner.load_state_dict(weights)

In [ ]:

# agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
#                                         study_name='pearl-2024-11-12-hp-search',
#                                         action_space_dim=2,
#                                         observation_space_dim=30)
# agent=load_agent_weights(agent,weight_path=agent_path)

In [23]:
profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
print(f"Testing Return AVG Profit: {profit}, AVG Number of Trades: {n_trades}")

Testing Return AVG Profit: 1296.970877901161, AVG Number of Trades: 37.15
